In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import tqdm
import warnings
from pyecharts import Bar , Timeline
warnings.filterwarnings('ignore')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lml/loader.py:64: UserWarning: Deprecated! since version 0.0.3. Please use scan_plugins_regex!
  "Deprecated! since version 0.0.3. Please use scan_plugins_regex!"
pyecharts_snapshot is abscent or cannot be imported
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lml/utils.py", line 42, in do_import
    return _do_import(plugin_module_name)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lml/utils.py", line 51, in _do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [2]:
df = pd.read_csv("count_df.csv", encoding='gbk')
df = df.sort_values(by = ['刷卡站点','月份', '日' , '时', '第x个五分钟'])

hours = []
minuteOfHours = []
for hour in range(24):
    for minuteOfHour in range(12):
        hours.append(hour)
        minuteOfHours.append(minuteOfHour)
        
test_df = pd.DataFrame({"时": hours , "第x个五分钟": minuteOfHours })

stations = set(df['刷卡站点'])

In [3]:
for station in tqdm.tqdm(stations):
    mask = df['刷卡站点'] == station
    temp_df = df[mask]
    X = temp_df[['时', '第x个五分钟']]
    y = temp_df['进出站状态']
    clf = KNeighborsRegressor()
    clf.fit(X,y)
    test_df = pd.DataFrame({"时": hours , "第x个五分钟": minuteOfHours })
    y_pred = clf.predict(test_df)
    pred_df = test_df
    pred_df['客流量'] = y_pred
    pred_df.to_csv("./knn_outputs/{}.csv".format(station), index = None , encoding = 'gbk')
    



100%|██████████| 313/313 [00:05<00:00, 57.83it/s]


In [4]:
for station in tqdm.tqdm(stations):
    mask = df['刷卡站点'] == station
    temp_df = df[mask]
    X = temp_df[['时', '第x个五分钟']]
    y = temp_df['进出站状态']
    clf = DecisionTreeRegressor()
    clf.fit(X,y)
    test_df = pd.DataFrame({"时": hours , "第x个五分钟": minuteOfHours })
    y_pred = clf.predict(test_df)
    pred_df = test_df
    pred_df['客流量'] = y_pred
    pred_df.to_csv("./dt_outputs/{}.csv".format(station), index = None , encoding = 'gbk')
    



100%|██████████| 313/313 [00:04<00:00, 63.06it/s]


In [5]:
DFs = []
for fileName in os.listdir('./dt_outputs/'):
    temp_df = pd.read_csv('./dt_outputs/' + fileName , encoding='gbk')
    station = fileName.split('.')[0]
    temp_df['站点'] = station
    DFs.append(temp_df)
pred_df = pd.concat(DFs)
del DFs

timeline = Timeline(is_auto_play=True, timeline_bottom=0,width = 1400 , height = 700)

for hour in range(24):
    for minuteOfHour in range(12):
        mask = (pred_df['时'] == hour)  & (pred_df['第x个五分钟'] == minuteOfHour)
        temp_df = temp_df.sort_values('站点')
        temp_df = pred_df[mask]
        attrs = temp_df['站点']
        values = temp_df['客流量']
        
        bar = Bar("24小时上海地铁客流量", "以每5分钟为统计单位" , width = 1400 , height = 600)
        bar.add("", attrs, values)
        timeline.add(bar, '{}时 ：{}～{}分钟'.format(hour, 5*minuteOfHour, 5*minuteOfHour+5 ))
timeline.render('dt_24小时上海地铁客流量.html')       